In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn import metrics
import pandas_datareader.data as web 
from time import time
from sklearn.model_selection import KFold

#Read stock data use pandas_datareader.data from web
#Get Stock Data

In [51]:
stk_tickers = ['MSFT', 'IBM', 'GOOGL']
ccy_tickers = ['DEXJPUS', 'DEXUSUK']
idx_tickers = ['SP500', 'DJIA', 'VIXCLS']
stk_data = web.DataReader(stk_tickers, 'yahoo')
ccy_data = web.DataReader(ccy_tickers, 'fred')
idx_data = web.DataReader(idx_tickers, 'fred')

In [ ]:
#Select columns
base = stk_data.loc[:, ('Adj Close', 'MSFT')]
X1 = stk_data.loc[:, ('Adj Close', ('GOOGL', 'IBM'))]
X2 = ccy_data
X3 = idx_data
print(X1)
print(X2)
print(X2)

Attributes   Adj Close            
Symbols          GOOGL         IBM
Date                              
2017-09-22   47.162998  109.384460
2017-09-25   46.714001  109.942162
2017-09-26   46.871498  110.462212
2017-09-27   47.994999  109.783897
2017-09-28   48.240501  109.783897
...                ...         ...
2022-09-16  102.800003  127.269997
2022-09-19  103.070000  127.730003
2022-09-20  101.139999  126.300003
2022-09-21   99.279999  124.930000
2022-09-22  100.181801  125.344101

[1259 rows x 2 columns]
            DEXJPUS  DEXUSUK
DATE                        
2017-09-25   111.64   1.3457
2017-09-26   112.16   1.3422
2017-09-27   112.76   1.3401
2017-09-28   112.66   1.3435
2017-09-29   112.64   1.3402
...             ...      ...
2022-09-12   142.41   1.1701
2022-09-13   144.30   1.1526
2022-09-14   142.93   1.1564
2022-09-15   143.57   1.1472
2022-09-16   143.04   1.1419

[1300 rows x 2 columns]
            DEXJPUS  DEXUSUK
DATE                        
2017-09-25   111.64   1.3

In [ ]:
#Standardized data (X1, X2, X3) with kept index (date)
standard_scaler = preprocessing.StandardScaler()
sX1 = standard_scaler.fit_transform(X1.values)
sX2 = standard_scaler.fit_transform(X2.values)
sX3 = standard_scaler.fit_transform(X3.values)
X1 = pd.DataFrame(index = X1.index,data = sX1,columns=X1.columns)
X2 = pd.DataFrame(index = X2.index,data = sX2,columns=X2.columns)
X3 = pd.DataFrame(index = X3.index,data = sX3,columns=X3.columns)


#Calculate ความแตกต่างของค่า ราคา 'Adj Close', 'MSFT’)ย้อนหลัง backHistory วัน

In [ ]:

backHistory = [30, 45, 60, 90, 180, 240] #-> ทดลองหยิบ 3 ค่า 3 รูปแบบ เพื่อดูระยะเวลาการดูค่าข้อมูลย้อนหลงัหลายๆแบบและเปรียบเทียบ MSE
BH1, BH2, BH3 = backHistory[1], backHistory[3], backHistory[4]
return_period = 1
Y = base.shift(-return_period)
X4_BH1 = base.diff( BH1).shift( - BH1)
X4_BH2 = base.diff( BH2).shift( - BH2)
X4_BH3 = base.diff( BH3).shift( - BH3)

X4 = pd.concat([X4_BH1, X4_BH2, X4_BH3], axis=1)
X4.columns = ['MSFT_3DT', 'MSFT_6DT', 'MSFT_12DT']
X4 = pd.DataFrame(standard_scaler.fit_transform(X4.values), index = X4.index,columns=X4.columns)

In [ ]:
# Forming Dataset
X = pd.concat([X1, X2, X3, X4], axis=1)
dataset = pd.concat([Y, X], axis=1)
dataset.dropna(inplace=True)
dataset.describe()


,"(Adj Close, MSFT)","(Adj Close, GOOGL)","(Adj Close, IBM)",DEXJPUS,DEXUSUK,SP500,DJIA,VIXCLS,MSFT_3DT,MSFT_6DT,MSFT_12DT
count,1061.000000,1061.000000,1061.000000,1061.000000,1061.000000,1061.000000,1061.000000,1061.000000,1061.000000,1061.000000,1061.000000
mean,165.735348,-0.204633,-0.222742,-0.338575,0.130736,-0.197839,-0.184988,-0.101468,0.126916,0.151276,0.001492
std,73.249716,0.921306,0.874408,0.357050,0.913578,0.931764,0.950400,1.034052,0.890119,0.876217,0.994522
min,68.838753,-1.158607,-3.287890,-1.214892,-2.751596,-1.663539,-2.423450,-1.308869,-3.833631,-3.764976,-3.678837
25%,102.691040,-0.861345,-0.698625,-0.593148,-0.461011,-0.901739,-0.865044,-0.779002,-0.269230,-0.232088,-0.412128
50%,140.447113,-0.646913,-0.248756,-0.326500,0.052543,-0.590221,-0.510803,-0.362019,0.091052,0.087653,0.053786
75%,214.606934,0.140018,0.261709,-0.081964,0.885793,0.416846,0.347160,0.232355,0.532599,0.590283,0.626045
max,340.882782,2.107258,1.898291,0.554088,2.077850,2.055378,1.992546,7.163257,2.578360,2.211266,2.422473


In [ ]:
# Assign X, Y (drop datetime index)
Y = pd.DataFrame(dataset[dataset.columns[0]])
X = pd.DataFrame(dataset[dataset.columns[1:]])
print(Y)
print(X)

             Adj Close
                  MSFT
2017-09-25   68.838753
2017-09-26   69.393127
2017-09-27   69.411934
2017-09-28   69.994522
2017-09-29   70.107262
...                ...
2021-12-28  339.730377
2021-12-29  337.117432
2021-12-30  334.136902
2022-01-03  326.874359
2022-01-04  314.326324

[1061 rows x 1 columns]
            (Adj Close, GOOGL)  (Adj Close, IBM)   DEXJPUS   DEXUSUK  \
2017-09-25           -1.158607         -0.510038 -0.028635  0.589904   
2017-09-26           -1.153619         -0.462036  0.039003  0.530386   
2017-09-27           -1.118039         -0.524646  0.117046  0.494676   
2017-09-28           -1.110264         -0.524646  0.104039  0.552493   
2017-09-29           -1.096155         -0.564994  0.101437  0.496376   
...                        ...               ...       ...       ...   
2021-12-28            2.007452          1.144712  0.375889  0.547391   
2021-12-29            2.006439          1.208785  0.404505  0.620513   
2021-12-30            1.9920

In [ ]:
X.corr()

,"(Adj Close, GOOGL)","(Adj Close, IBM)",DEXJPUS,DEXUSUK,SP500,DJIA,VIXCLS,MSFT_3DT,MSFT_6DT,MSFT_12DT
"(Adj Close, GOOGL)",1.000000,0.486192,0.094845,0.483430,0.982740,0.952258,0.047198,0.040460,-0.059934,-0.370779
"(Adj Close, IBM)",0.486192,1.000000,0.236112,0.588217,0.506262,0.589115,-0.532156,-0.002491,-0.047912,-0.375558
DEXJPUS,0.094845,0.236112,1.000000,0.077120,0.011478,0.057563,-0.407793,-0.287386,-0.428954,-0.575826
DEXUSUK,0.483430,0.588217,0.077120,1.000000,0.486225,0.528740,-0.304232,0.063434,0.026694,-0.199002
SP500,0.982740,0.506262,0.011478,0.486225,1.000000,0.982879,0.002491,0.011353,-0.035838,-0.281116
DJIA,0.952258,0.589115,0.057563,0.528740,0.982879,1.000000,-0.126383,-0.007451,-0.029897,-0.270837
VIXCLS,0.047198,-0.532156,-0.407793,-0.304232,0.002491,-0.126383,1.000000,0.284879,0.338162,0.259841
MSFT_3DT,0.040460,-0.002491,-0.287386,0.063434,0.011353,-0.007451,0.284879,1.000000,0.654932,0.389500
MSFT_6DT,-0.059934,-0.047912,-0.428954,0.026694,-0.035838,-0.029897,0.338162,0.654932,1.000000,0.671310
MSFT_12DT,-0.370779,-0.375558,-0.575826,-0.199002,-0.281116,-0.270837,0.259841,0.389500,0.671310,1.000000


In [ ]:
lower = pd.DataFrame(np.tril(X.corr(),-1),columns = X.columns)
to_drop = [column for column in lower.columns if any(lower[column] > 0.9)]
X.drop(columns=to_drop,inplace=True)
to_drop

[('Adj Close', 'GOOGL'), 'SP500']

In [ ]:
X

,"(Adj Close, IBM)",DEXJPUS,DEXUSUK,DJIA,VIXCLS,MSFT_3DT,MSFT_6DT,MSFT_12DT
2017-09-25,-0.510038,-0.028635,0.589904,-1.525064,-1.185618,0.238253,0.144326,-0.149111
2017-09-26,-0.462036,0.039003,0.530386,-1.527919,-1.190225,0.154568,-0.000744,-0.133429
2017-09-27,-0.524646,0.117046,0.494676,-1.514242,-1.224782,0.167771,0.104526,-0.184927
2017-09-28,-0.524646,0.104039,0.552493,-1.504422,-1.261642,0.171579,0.037751,-0.165385
2017-09-29,-0.564994,0.101437,0.496376,-1.498628,-1.266250,-0.034746,-0.162466,-0.182206
...,...,...,...,...,...,...,...,...
2021-12-28,1.144712,0.375889,0.547391,1.895183,-0.341285,-2.969524,-3.243323,-3.640278
2021-12-29,1.208785,0.404505,0.620513,1.917113,-0.409246,-3.356701,-3.682247,-3.678837
2021-12-30,1.258618,0.430520,0.663026,1.895151,-0.365474,-3.833631,-3.377000,-3.609747
2022-01-03,1.448163,0.443527,0.610310,1.940500,-0.449562,-3.022997,-3.764976,-3.639475


In [ ]:
# Train / Test Preparation (try 2 Option)Option#1
from math import ceil, floor
test_size = floor(0.3 * len( X ))
train_size = floor(0.7 * len( X ))
print(f'test size = {test_size}\ntrain size = {train_size}')
X_train, X_test = X[0:train_size], X[train_size:len(X)]
Y_train, Y_test = Y[0:train_size], Y[train_size:len(X)]
# Option #2
# X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=0.3, random_state=seed)

test size = 318
train size = 742


#Create Model List and Parameter Dictionary

In [ ]:
# Create Model List
regression = { 'LR': LinearRegression(), 'SVR': SVR(), }
# Create Parameter Dictionary for Linear Regression
fit_intercept = [True, False]
normalize = [True, False]
params_LR = dict( fit_intercept = fit_intercept, normalize = normalize)
print(params_LR)

{'fit_intercept': [True, False], 'normalize': [True, False]}


#Create Parameter Dictionary for SVR

In [ ]:
kernel = ['linear', 'rbf', 'poly']
C_list = [10, 100]
ep_list = [0.1, 1, 5]
gamma = [0.01, 0.1]
degree = [2, 3]       #3x2x3x2x2 = 72
params_SVR = dict( kernel = kernel, C = C_list, epsilon = ep_list, gamma = gamma, degree = degree )



# 6.2 GridSearchCV() -> (a)

In [ ]:
for EST in regression:
    model = regression[EST]
    if (EST == 'LR'):
        params = params_LR
    else:
        params = params_SVR

    grid = GridSearchCV( 
    estimator=model, 
    n_jobs = 8,
    verbose = 10,
    cv = 3,
    scoring = 'neg_mean_squared_error',
    param_grid = params )
    grid_result = grid.fit(X_train, Y_train)

# Show Best Parameters for both models

In [ ]:
print('Best params: ',grid_result.best_params_)
print('Best score: ', grid_result.best_score_)

NameError: name 'grid_result' is not defined

# Show Score for each parameter combination for both model

In [ ]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Display Mean, std, params

#6.2 Create Model List and Parameter Dictionary

In [ ]:
# Create Model List
regression = { 'LR': LinearRegression(), 'SVR': SVR(), }

In [ ]:
# Create Parameter Dictionary for Linear Regression
fit_intercept = [True, False]
normalize = [True, False]
params_LR = dict( fit_intercept = fit_intercept, normalize = normalize)

In [ ]:
# Create Parameter Dictionary for SVR
kernel = ['linear', 'rbf', 'poly']
C_list = list(np.linspace(0.1, 150, 5, dtype = float))
ep_list = list(np.linspace(0.1, 1, 5, dtype = float))
gamma = list(np.linspace(0.01, 0.1, 5, dtype = float))
degree = [2, 3]
params_SVR = dict( kernel = kernel, C = C_list, epsilon = ep_list, gamma = gamma, degree = degree )

# Show scatter plot compare y_test vs each model prediction

#6.3 RandomizedSearchCV() -> (a)

In [ ]:
for EST in regression:
    model = regression[EST]
    if (EST == 'LR'):
        params = params_LR
    else:
        params = params_SVR

    grid_rand = RandomizedSearchCV( estimator=model, n_jobs = 1,

    verbose = 10,
    cv = k,
    scoring = 'neg_mean_squared_error',
    param_distribution = params )

grid_rand_result = grid_rand.fit(X_train, Y_train)

In [ ]:
# Show Best Parameters for both models
print('Best params: ',grid_rand_result.best_params_)
print('Best score: ', grid_rand_result.best_score_)

In [ ]:
# Show Score for each parameter combination for both model
means = grid_rand_result.cv_results_['mean_test_score']
stds = grid_rand_result.cv_results_['std_test_score']
params = grid_rand_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Display Mean, std, params

# Show scatter plot compare y_test vs each model prediction